In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fastai.text import *
from pprint import pprint

In [2]:
df = pd.read_csv('/media/winstonfan/Workspace/MyDoc/Interviews/DigitalVictoria/code/COVID-Q/final_master_dataset.csv')

In [3]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Category    366
Question      1
Answers     763
dtype: int64

## 1. Construct the word dataset for language model

In [4]:
df_word = df[['Question']]

In [5]:
# Check None records
null_columns=df_word.columns[df_word.isnull().any()]
df_word[null_columns].isnull().sum()

Question    1
dtype: int64

In [6]:
df_word = df_word.dropna(subset=['Question'])
df_word.columns=['text']
df_word.tail()

,text
1606,why are churches closed due to covid
1607,is it safe to have play rehearsals during covid
1608,where can i buy toilet paper considering most ...
1609,will i likely catch the covid virus in america...
1610,what are your best tips for entertaining young...


In [9]:
# save the data set for use later
# word_df.to_csv('word_ds.csv', index=False)

### 2. Construct the dataset for text classification model

In [8]:
df_questions = df[['Category', 'Question']]

df_questions = df_questions.dropna(subset=['Category', 'Question'])

df_questions.columns = ['label','text']
df_questions.tail()

,label,text
1240,Reporting - Media Coverup,is the media hiding what is really going on in...
1241,Treatment - Quinine,can quinine be taken to fight this covid virus
1242,Treatment - Quinine,why is quinine effective in curing covid
1243,Other - Food Shortages,could we run out of things to eat
1244,Other - Food Shortages,do i need to stock up on food


In [12]:
df_questions.shape

(1244, 2)

In [13]:
categories = df_questions.label.unique()
categories.shape

(208,)

In [22]:
categories = np.sort(categories)
df_categories = pd.DataFrame(categories)
df_categories['index'] = df_categories.index
df_categories.columns = ['label', 'cid']
df_categories.head()

,label,cid
0,Comparison - Black Death,0
1,Comparison - COVID and HIV,1
2,Comparison - COVID versus Flu Diagnosis,2
3,Comparison - Difference from Flu,3
4,Comparison - Earlier Existence,4


It doesn't matter whether we sort the categories as long as we keep the relationship between category name and its index, so that we can retrieve the correct category name later by its index

In [57]:
df_categories.to_csv('labels.csv', index=False)

In [35]:
a = df_categories[df_categories['label'] == 'Comparison - COVID versus Flu Diagnosis']
if len(a) > 0:
    print('wola~!')
else:
    print('soga~!')

wola~!


In [47]:
a['cid'].values[0]

2

In [58]:
def getCategoryId(label):
    if len(label) == 0:
        raise ValueError('getCategoryId() gets an invalid value')
    
    l = str.strip(label)
    label_cid = df_categories[df_categories['label'] == l]
    if len(label_cid) == 0:
        raise ValueError('getCategoryId() does not find a category for {}'.format(label))
        
    index = label_cid['cid'].values[0]
    
    return index

# getCategoryId('Comparison - Difference from Flu')
df_questions['cid'] = df_questions['label'].apply(lambda label: getCategoryId(label))
df_questions.tail()

,label,text,cid
1240,Reporting - Media Coverup,is the media hiding what is really going on in...,91
1241,Treatment - Quinine,can quinine be taken to fight this covid virus,207
1242,Treatment - Quinine,why is quinine effective in curing covid,207
1243,Other - Food Shortages,could we run out of things to eat,60
1244,Other - Food Shortages,do i need to stock up on food,60


In [60]:
df_questions[df_questions['cid']==174]

,label,text,cid
1159,Transmission - Countries with no Cases,is there a country whose people are not expose...,174
1160,Transmission - Countries with no Cases,what countries are still safe from covid,174


In [61]:
df_questions.to_csv('questions_ds.csv', index=False)